# Data Cleaning Listado de Listado de Beneficiarios 2019-2022

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

## Importante leer para entender

En este Notebook se trabaja con 3 listados importantes:

1. Beneficiarios2019_2022 -> Dataset con el listado de TODOS los beneficiarios.
2. Estados_Beneficiarios_2019_2022 -> Dataset sólo con los estados *únicos* encontrados en el dataset anterior.
3. Diccionario -> Emparejamiento entre ambos listados de beneficiarios
4. Dataset_Inegi -> Catálogo obtenido de Inegi

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
def load_datasets(directory):
    # Get a list of all CSV files in the directory
    csv_files = glob.glob(os.path.join(directory, '*.csv'))

    # Read each CSV file and store the DataFrame in a list
    dataframes = [pd.read_csv(file, encoding='utf-8', skiprows=0) for file in csv_files]

    # Print the number of rows for each DataFrame
    for i, df in enumerate(dataframes):
        print(f"Number of rows in DataFrame {i+1}: {df.shape[0]}")

    # Calculate the sum of rows in each individual dataset
    individual_row_sum = sum([df.shape[0] for df in dataframes])

    # Concatenate all DataFrames in the list
    merged_df = pd.concat(dataframes, join='inner', ignore_index=True)

    return merged_df, individual_row_sum


def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text) #Remove what is in between - -
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura de los datos

### Lectura del dataset del INEGI

In [3]:
path_dataset_inegi = '../../data/dataset_inegi_clean.csv'
dataset_inegi_clean = pd.read_csv(path_dataset_inegi, encoding='utf-8', dtype={'CVE_ENT': str, 'CVE_MUN': str, 'CVE_LOC': str})

### Lectura del listado de Beneficiarios 2019-2022

In [4]:
listado_beneficiarios, sumOfRows = load_datasets("../../data/productores_beneficiarios 2019-2022")

Number of rows in DataFrame 1: 826385
Number of rows in DataFrame 2: 394986
Number of rows in DataFrame 3: 369124
Number of rows in DataFrame 4: 278550


In [5]:
nan_rows = listado_beneficiarios.isna()

In [6]:
sumOfRows

1869045

In [7]:
listado_beneficiarios.head()

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,FECHA,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA
0,ABAC NERI JESUS CARLOS,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,PUEBLO VIEJO,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,2022-06-26,12244.18,Fertilizantes,AGRARIA,AGRARIA,PV2022
1,ABAD ALARCON ELIAZAR,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,PUEBLO VIEJO,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,2022-06-26,6122.09,Fertilizantes,AGRARIA,AGRARIA,PV2022
2,ABAD ALARCON EZEQUIEL,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,AGUA ZARCA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,2022-07-12,12244.18,Fertilizantes,AGRARIA,AGRARIA,PV2022
3,ABAD ALARCON FULGENCIO,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,AGUA ZARCA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,2022-07-12,12244.18,Fertilizantes,AGRARIA,AGRARIA,PV2022
4,ABAD ALARCON ROMALDA,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,YERBASANTITA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,2022-06-20,12244.18,Fertilizantes,AGRARIA,AGRARIA,PV2022


In [8]:
listado_beneficiarios.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA'],
      dtype='object')

# 2. Limpieza de datos.

### 2.1 Beneficiarios 2019-2022

### Creación de Estados_Beneficiarios2019-2022
Este dataset es una versión de Estados_beneficiarios2019-2022 pero más ligera y sin repeticiones.

In [9]:
listado_beneficiarios.dtypes

BENEFICIARIO        object
ZONA                object
ENTIDAD             object
MUNICIPIO           object
LOCALIDAD           object
ESTRATIFICACIÓN     object
PROGRAMA            object
COMPONENTE          object
SUBCOMPONENTE       object
PRODUCTO            object
FECHA               object
MONTO FEDERAL      float64
APOYO               object
ACTIVIDAD           object
ESLABÓN             object
CICLO AGRÍCOLA      object
dtype: object

In [10]:
descriptive_stats = listado_beneficiarios.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

                     count  unique                           top     freq  \
BENEFICIARIO       1869037  817428             PEREZ PEREZ JUAN       111   
ZONA               1869033       8                   CENTRO PAÍS  1424972   
ENTIDAD            1869033      10                      GUERRERO  1279305   
MUNICIPIO          1869033    1025            CHILAPA DE ÁLVAREZ    60148   
LOCALIDAD          1869033    9979                     TECOANAPA    10820   
ESTRATIFICACIÓN    1869033       7                          Alto  1253286   
PROGRAMA           1869033       1                 FERTILIZANTES  1869033   
COMPONENTE         1869033       1                 FERTILIZANTES  1869033   
SUBCOMPONENTE      1869033       1                 FERTILIZANTES  1869033   
PRODUCTO           1869033       7  Cultivo de maíz grano blanco  1132720   
FECHA              1869033     815                    15/07/2019    17568   
MONTO FEDERAL    1869033.0     NaN                           NaN      NaN   

In [11]:
listado_beneficiarios = listado_beneficiarios.drop(columns=['PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE', 'APOYO', 'ACTIVIDAD', 'ESLABÓN'])

In [12]:
rows_with_nan = listado_beneficiarios[listado_beneficiarios.isna().any(axis=1)]

In [13]:
rows_with_nan

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PRODUCTO,FECHA,MONTO FEDERAL,CICLO AGRÍCOLA
826382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
826383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
826384,"""Nota: El Sistema de Rendición de Cuentas sobr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1221368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1221369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1221370,"""Nota: El Sistema de Rendición de Cuentas sobr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1590492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1590493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1590494,"""Nota: El Sistema de Rendición de Cuentas sobr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1869042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# QUE HACEMOS CON ESTAS NOTAS?

In [14]:
listado_beneficiarios = listado_beneficiarios.drop(rows_with_nan.index)

In [15]:
listado_beneficiarios.isna().sum()

BENEFICIARIO       0
ZONA               0
ENTIDAD            0
MUNICIPIO          0
LOCALIDAD          0
ESTRATIFICACIÓN    0
PRODUCTO           0
FECHA              0
MONTO FEDERAL      0
CICLO AGRÍCOLA     0
dtype: int64

In [16]:
listado_beneficiarios.head()

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PRODUCTO,FECHA,MONTO FEDERAL,CICLO AGRÍCOLA
0,ABAC NERI JESUS CARLOS,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,PUEBLO VIEJO,Alto,Cultivo de maíz grano blanco,2022-06-26,12244.18,PV2022
1,ABAD ALARCON ELIAZAR,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,PUEBLO VIEJO,Alto,Cultivo de maíz grano blanco,2022-06-26,6122.09,PV2022
2,ABAD ALARCON EZEQUIEL,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,AGUA ZARCA,Alto,Cultivo de maíz grano blanco,2022-07-12,12244.18,PV2022
3,ABAD ALARCON FULGENCIO,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,AGUA ZARCA,Alto,Cultivo de maíz grano blanco,2022-07-12,12244.18,PV2022
4,ABAD ALARCON ROMALDA,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,YERBASANTITA,Alto,Cultivo de maíz grano blanco,2022-06-20,12244.18,PV2022


In [17]:
Estados_beneficiarios2019_2022 = listado_beneficiarios[['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD']]

In [18]:
Estados_beneficiarios2019_2022 = Estados_beneficiarios2019_2022.drop_duplicates()

In [19]:
# Estandarizamos la limpieza de los datos
Estados_beneficiarios2019_2022['ESTADO_c_benef'] = Estados_beneficiarios2019_2022['ENTIDAD'].apply(clean_text)
Estados_beneficiarios2019_2022['MUNICIPIO_c_benef'] = Estados_beneficiarios2019_2022['MUNICIPIO'].apply(clean_text)
Estados_beneficiarios2019_2022['LOCALIDAD_c_benef'] = Estados_beneficiarios2019_2022['LOCALIDAD'].apply(clean_text)


In [20]:
Estados_beneficiarios2019_2022.shape

(13219, 6)

In [21]:
Estados_beneficiarios2019_2022['KEY_benef'] = Estados_beneficiarios2019_2022['ESTADO_c_benef'].astype(str) + '-' + Estados_beneficiarios2019_2022[
    'MUNICIPIO_c_benef'].astype(str)

Estados_beneficiarios2019_2022['KEY_benef_localidad'] = Estados_beneficiarios2019_2022['ESTADO_c_benef'].astype(str) + '-' + Estados_beneficiarios2019_2022[
    'MUNICIPIO_c_benef'].astype(str)

Estados_beneficiarios2019_2022.head()


,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTADO_c_benef,MUNICIPIO_c_benef,LOCALIDAD_c_benef,KEY_benef,KEY_benef_localidad
0,GUERRERO,COYUCA DE BENÍTEZ,PUEBLO VIEJO,guerrero,coyuca de benitez,pueblo viejo,guerrero-coyuca de benitez,guerrero-coyuca de benitez
2,GUERRERO,COYUCA DE BENÍTEZ,AGUA ZARCA,guerrero,coyuca de benitez,agua zarca,guerrero-coyuca de benitez,guerrero-coyuca de benitez
4,GUERRERO,COYUCA DE BENÍTEZ,YERBASANTITA,guerrero,coyuca de benitez,yerbasantita,guerrero-coyuca de benitez,guerrero-coyuca de benitez
5,GUERRERO,CUTZAMALA DE PINZÓN,LIMÓN GRANDE,guerrero,cutzamala de pinzon,limon grande,guerrero-cutzamala de pinzon,guerrero-cutzamala de pinzon
6,OAXACA,SAN JUAN GUICHICOVI,SAN JUAN GUICHICOVI,oaxaca,san juan guichicovi,san juan guichicovi,oaxaca-san juan guichicovi,oaxaca-san juan guichicovi


In [22]:
# Valores únicos y la cantidad de cada columna
# Obtener estadísticas descriptivas para todas las variables

descriptive_stats = Estados_beneficiarios2019_2022.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

                     count unique                         top  freq
ENTIDAD              13219     10                    GUERRERO  5259
MUNICIPIO            13219   1025           COYUCA DE CATALÁN   215
LOCALIDAD            13219   9979                  BUENAVISTA    37
ESTADO_c_benef       13219     10                    guerrero  5259
MUNICIPIO_c_benef    13219   1025           coyuca de catalan   215
LOCALIDAD_c_benef    13219   9969                  buenavista    37
KEY_benef            13219   1033  guerrero-coyuca de catalan   215
KEY_benef_localidad  13219   1033  guerrero-coyuca de catalan   215


In [23]:
Estados_beneficiarios2019_2022 = Estados_beneficiarios2019_2022.drop_duplicates(subset='KEY_benef')

In [24]:
Estados_beneficiarios2019_2022.shape

(1033, 8)

### 2.1.2 Reducción de inegi en base a estados beneficiarios.

In [25]:
unique_entidad_list_benef = Estados_beneficiarios2019_2022['ESTADO_c_benef'].unique().tolist()
print(unique_entidad_list_benef)

['guerrero', 'oaxaca', 'puebla', 'morelos', 'chiapas', 'durango', 'tlaxcala', 'nayarit', 'zacatecas', 'nacional']


# Que es nacional?

In [26]:
dataset_inegi_clean.shape

(299568, 9)

In [27]:
dataset_inegi_clean_filtered = dataset_inegi_clean[dataset_inegi_clean['Entidad_c_inegi'].isin(unique_entidad_list_benef)]

In [28]:
unique_entidad_list_inegi = dataset_inegi_clean_filtered['Entidad_c_inegi'].unique().tolist()
print(unique_entidad_list_inegi)

['chiapas', 'durango', 'guerrero', 'morelos', 'nayarit', 'oaxaca', 'puebla', 'tlaxcala', 'zacatecas']


In [29]:
dataset_inegi_clean_filtered.shape

(83449, 9)

In [30]:
dataset_inegi_clean = dataset_inegi_clean_filtered

### 2.1 Inegi dataset

In [31]:
dataset_inegi_clean.columns

Index(['CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi', 'CVE_LOC',
       'Localidad_inegi', 'Entidad_c_inegi', 'Municipio_c_inegi',
       'Localidad_c_inegi'],
      dtype='object')

In [32]:
# Revisamos la cantidad de filas y columnas del dataset
print("Shape of dataset_inegi: ", dataset_inegi_clean.shape)

dataset_inegi_uniquemun = dataset_inegi_clean.drop_duplicates(subset=['CVE_MUN', 'Municipio_inegi'], keep='first')
print("Shape of dataset_inegi_uniquemun: ", dataset_inegi_uniquemun.shape)

dataset_inegi_uniqueloc = dataset_inegi_clean.drop_duplicates(subset=['CVE_LOC', 'Localidad_inegi'], keep='first')
print("Shape of dataset_inegi_uniqueloc: ", dataset_inegi_uniqueloc.shape)

Shape of dataset_inegi:  (83449, 9)
Shape of dataset_inegi_uniquemun:  (1209, 9)
Shape of dataset_inegi_uniqueloc:  (81569, 9)


In [33]:
'''
# Group by 'Localidad_inegi' and aggregate unique 'Municipio_inegi' values
localidad_municipio = dataset_inegi_clean.groupby('Localidad_inegi')['Municipio_inegi'].unique()

# Create a summary string for each 'localidad_inegi' showing count and municipio names
localidad_municipio_summary = localidad_municipio.apply(lambda x: f"{len(x)} - {', '.join(x)}")

# Filter to find 'localidad_inegi' with more than one 'municipio_inegi'
multiple_municipios_summary = localidad_municipio_summary[localidad_municipio.apply(len) > 1]

print("Localidad_inegi with multiple municipio_inegi values and their counts:")
print(multiple_municipios_summary)
'''

'\n# Group by \'Localidad_inegi\' and aggregate unique \'Municipio_inegi\' values\nlocalidad_municipio = dataset_inegi_clean.groupby(\'Localidad_inegi\')[\'Municipio_inegi\'].unique()\n\n# Create a summary string for each \'localidad_inegi\' showing count and municipio names\nlocalidad_municipio_summary = localidad_municipio.apply(lambda x: f"{len(x)} - {\', \'.join(x)}")\n\n# Filter to find \'localidad_inegi\' with more than one \'municipio_inegi\'\nmultiple_municipios_summary = localidad_municipio_summary[localidad_municipio.apply(len) > 1]\n\nprint("Localidad_inegi with multiple municipio_inegi values and their counts:")\nprint(multiple_municipios_summary)\n'

Por aquí existen localidades con nombres de fechas lo cual debe ser común en México ya que se repiten en diversas ocasiones.

In [34]:
'''
# Filter the dataset for rows where 'Municipio' equals '10 De Abril'
filtered_dataset = dataset_inegi_clean[dataset_inegi_clean['Localidad_inegi'] == '10 de Abril']

# Display the filtered dataset
print(filtered_dataset)
'''

"\n# Filter the dataset for rows where 'Municipio' equals '10 De Abril'\nfiltered_dataset = dataset_inegi_clean[dataset_inegi_clean['Localidad_inegi'] == '10 de Abril']\n\n# Display the filtered dataset\nprint(filtered_dataset)\n"

A partir de aquí seguimos trabajando con el listado de Estados y Municipios limpio de Inegi (sin repetir) "dataset_inegi_uniquemun".

In [35]:
# Revisamos las primeras filas del dataset con las columnas seleccionadas
dataset_inegi_uniquemun.head()

,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,CVE_LOC,Localidad_inegi,Entidad_c_inegi,Municipio_c_inegi,Localidad_c_inegi
33056,07,Chiapas,001,Acacoyagua,1,Acacoyagua,chiapas,acacoyagua,acacoyagua
33200,07,Chiapas,002,Acala,1,Acala,chiapas,acala,acala
33352,07,Chiapas,003,Acapetahua,1,Acapetahua,chiapas,acapetahua,acapetahua
33602,07,Chiapas,004,Altamirano,1,Altamirano,chiapas,altamirano,altamirano
33795,07,Chiapas,005,Amatán,1,Amatán,chiapas,amatan,amatan


In [36]:
dataset_inegi_uniquemun.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1209 entries, 33056 to 299526
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CVE_ENT            1209 non-null   object
 1   Entidad_inegi      1209 non-null   object
 2   CVE_MUN            1209 non-null   object
 3   Municipio_inegi    1209 non-null   object
 4   CVE_LOC            1209 non-null   object
 5   Localidad_inegi    1209 non-null   object
 6   Entidad_c_inegi    1209 non-null   object
 7   Municipio_c_inegi  1209 non-null   object
 8   Localidad_c_inegi  1209 non-null   object
dtypes: object(9)
memory usage: 94.5+ KB


In [37]:
print("Los valores únicos en cada columna son:\n", dataset_inegi_uniquemun.nunique())

Los valores únicos en cada columna son:
 CVE_ENT                 9
Entidad_inegi           9
CVE_MUN               570
Municipio_inegi      1192
CVE_LOC                 2
Localidad_inegi      1197
Entidad_c_inegi         9
Municipio_c_inegi    1192
Localidad_c_inegi    1197
dtype: int64


In [38]:
print("Los valores únicos en cada columna son:\n", dataset_inegi_uniqueloc.nunique())

Los valores únicos en cada columna son:
 CVE_ENT                  9
Entidad_inegi            9
CVE_MUN                570
Municipio_inegi       1192
CVE_LOC               3286
Localidad_inegi      45780
Entidad_c_inegi          9
Municipio_c_inegi     1192
Localidad_c_inegi    45674
dtype: int64


#### 2.2.1 Estandarización de nombre de municipios

Con el fin de poder hacer un merge bajo los mismos nombres, hacemos una limpieza de los datos.

In [39]:
# Creamos una columna con la clave única por municipio

dataset_inegi_uniquemun['KEY_inegi'] = dataset_inegi_uniquemun['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_uniquemun[
    'Municipio_c_inegi'].astype(str)

dataset_inegi_uniquemun.head()

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_59794/2100087808.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_uniquemun['KEY_inegi'] = dataset_inegi_uniquemun['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_uniquemun[


,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,CVE_LOC,Localidad_inegi,Entidad_c_inegi,Municipio_c_inegi,Localidad_c_inegi,KEY_inegi
33056,07,Chiapas,001,Acacoyagua,1,Acacoyagua,chiapas,acacoyagua,acacoyagua,chiapas-acacoyagua
33200,07,Chiapas,002,Acala,1,Acala,chiapas,acala,acala,chiapas-acala
33352,07,Chiapas,003,Acapetahua,1,Acapetahua,chiapas,acapetahua,acapetahua,chiapas-acapetahua
33602,07,Chiapas,004,Altamirano,1,Altamirano,chiapas,altamirano,altamirano,chiapas-altamirano
33795,07,Chiapas,005,Amatán,1,Amatán,chiapas,amatan,amatan,chiapas-amatan


In [40]:
dataset_inegi_uniqueloc['KEY_inegi_localidad'] = dataset_inegi_uniqueloc['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_uniqueloc[
    'Municipio_c_inegi'].astype(str) + '-' + dataset_inegi_uniqueloc['Localidad_c_inegi'].astype(str)

dataset_inegi_uniqueloc.head()

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_59794/1997716209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_uniqueloc['KEY_inegi_localidad'] = dataset_inegi_uniqueloc['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_uniqueloc[


,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,CVE_LOC,Localidad_inegi,Entidad_c_inegi,Municipio_c_inegi,Localidad_c_inegi,KEY_inegi_localidad
33056,07,Chiapas,001,Acacoyagua,1,Acacoyagua,chiapas,acacoyagua,acacoyagua,chiapas-acacoyagua-acacoyagua
33057,07,Chiapas,001,Acacoyagua,2,Los Amates,chiapas,acacoyagua,los amates,chiapas-acacoyagua-los amates
33058,07,Chiapas,001,Acacoyagua,3,Arvin,chiapas,acacoyagua,arvin,chiapas-acacoyagua-arvin
33059,07,Chiapas,001,Acacoyagua,6,Los Cacaos,chiapas,acacoyagua,los cacaos,chiapas-acacoyagua-los cacaos
33060,07,Chiapas,001,Acacoyagua,7,El Castaño,chiapas,acacoyagua,el castano,chiapas-acacoyagua-el castano


In [41]:
dataset_inegi_uniquemun.drop_duplicates(subset=['KEY_inegi'], keep='first', inplace=True)

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_59794/1014247258.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_uniquemun.drop_duplicates(subset=['KEY_inegi'], keep='first', inplace=True)


In [42]:
dataset_inegi_uniqueloc.drop_duplicates(subset=['KEY_inegi_localidad'], keep='first', inplace=True)

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_59794/3976127218.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_uniqueloc.drop_duplicates(subset=['KEY_inegi_localidad'], keep='first', inplace=True)


In [43]:
dataset_inegi_uniquemun.shape

(1207, 10)

In [44]:
dataset_inegi_uniqueloc.shape

(73985, 10)

# 3. Diccionario de los datasets de INEGI Y LISTADO BENEFICIARIOS 2023

El objetivo de esta sección es crear un diccionario de códigos según INEGI para los municipios Listado_beneficiarios2023. Para ello haremos un Left join entre Estados_Beneficiarios2023 y dataset_inegi_uniquemun.

### 3.1 Creamos las columnas clave

Confirmamos que tengamos las columnas clave en ambos datasets.

In [45]:
Estados_beneficiarios2019_2022.columns 

Index(['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'ESTADO_c_benef',
       'MUNICIPIO_c_benef', 'LOCALIDAD_c_benef', 'KEY_benef',
       'KEY_benef_localidad'],
      dtype='object')

In [46]:
Estados_beneficiarios2019_2022.shape 

(1033, 8)

In [47]:
dataset_inegi_uniquemun.columns

Index(['CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi', 'CVE_LOC',
       'Localidad_inegi', 'Entidad_c_inegi', 'Municipio_c_inegi',
       'Localidad_c_inegi', 'KEY_inegi'],
      dtype='object')

In [48]:
dataset_inegi_uniquemun.shape 

(1207, 10)

### 3.1 Left join (Para crear diccionario)

Creamos el diccionario.

In [49]:
# Crear una función para encontrar la mejor coincidencia difusa con límites entre 90 y 100 de coincidencia
def fuzzy_merge_benef2019_2022(df_benef, df_inegi, key1, key2, threshold=50, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_inegi[key2].tolist()
    
    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_benef[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))


    # Crear una columna con las mejores coincidencias
    df_benef['best_match'] = [match[0] if match else None for match in matches]
    
    df_benef['match_score'] = [match[1] if match else None for match in matches]
    

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_benef, df_inegi, left_on= 'best_match', right_on=key2, how='left',
                         suffixes=('_benef', '_inegi'))
    
    return df_merged

In [50]:
diccionario = fuzzy_merge_benef2019_2022(Estados_beneficiarios2019_2022, dataset_inegi_uniquemun, 'KEY_benef', 'KEY_inegi')
# Mostrar el resultado
diccionario.columns

Index(['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'ESTADO_c_benef',
       'MUNICIPIO_c_benef', 'LOCALIDAD_c_benef', 'KEY_benef',
       'KEY_benef_localidad', 'best_match', 'match_score', 'CVE_ENT',
       'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi', 'CVE_LOC',
       'Localidad_inegi', 'Entidad_c_inegi', 'Municipio_c_inegi',
       'Localidad_c_inegi', 'KEY_inegi'],
      dtype='object')

In [51]:
'''
diccionario = fuzzy_merge_benef2019_2022(Estados_beneficiarios2019_2022, dataset_inegi_uniqueloc, 'KEY_benef', 'KEY_inegi_localidad')
'''

"\ndiccionario = fuzzy_merge_benef2019_2022(Estados_beneficiarios2019_2022, dataset_inegi_uniqueloc, 'KEY_benef', 'KEY_inegi_localidad')\n"

In [52]:
diccionario.shape

(1033, 20)

In [53]:
# Valores únicos y la cantidad de cada columna
# Obtener estadísticas descriptivas para todas las variables

descriptive_stats = diccionario.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

                      count unique                                top  freq  \
ENTIDAD                1033     10                             OAXACA   558   
MUNICIPIO              1033   1025                    EMILIANO ZAPATA     3   
LOCALIDAD              1033   1023                        LA TRINIDAD     3   
ESTADO_c_benef         1033     10                             oaxaca   558   
MUNICIPIO_c_benef      1033   1025                    emiliano zapata     3   
LOCALIDAD_c_benef      1033   1023                        la trinidad     3   
KEY_benef              1033   1033         guerrero-coyuca de benitez     1   
KEY_benef_localidad    1033   1033         guerrero-coyuca de benitez     1   
best_match             1033   1029  oaxaca-santiago pinotepa nacional     2   
match_score          1033.0    NaN                                NaN   NaN   
CVE_ENT                1033      9                                 20   557   
Entidad_inegi          1033      9                  

In [54]:
nan_rows = diccionario.isna()

In [55]:
nan_rows.sum()

ENTIDAD                0
MUNICIPIO              0
LOCALIDAD              0
ESTADO_c_benef         0
MUNICIPIO_c_benef      0
LOCALIDAD_c_benef      0
KEY_benef              0
KEY_benef_localidad    0
best_match             0
match_score            0
CVE_ENT                0
Entidad_inegi          0
CVE_MUN                0
Municipio_inegi        0
CVE_LOC                0
Localidad_inegi        0
Entidad_c_inegi        0
Municipio_c_inegi      0
Localidad_c_inegi      0
KEY_inegi              0
dtype: int64

In [56]:
diccionario.to_csv('../../data/diccionario_benef2019_2022.csv', index=False)

In [57]:
diccionario_benef2019_2022 = pd.read_csv('../../data/diccionario_benef2019_2022.csv')

In [58]:
# Step 1: Check data types
print(Estados_beneficiarios2019_2022['KEY_benef'].dtype)
print(diccionario_benef2019_2022['KEY_benef'].dtype)  # Replace 'YourKeyColumnName' with the actual column name

# Step 2: Trim whitespace and check formatting (example for strings)
key_benef_list = [str(key).strip().lower() for key in Estados_beneficiarios2019_2022['KEY_benef'].to_list()]  # Adjust if your keys are not strings
diccionario_keys = [str(key).strip().lower() for key in diccionario_benef2019_2022['KEY_benef'].tolist()]  # Adjust column name

keys_not_in_diccionario = [key for key in key_benef_list if key not in diccionario_keys]

# Count of keys not in diccionario
count_keys_not_in_diccionario = len(keys_not_in_diccionario)

print(f"Keys not in diccionario: {keys_not_in_diccionario}")
print(f"Count of keys not in diccionario: {count_keys_not_in_diccionario}")

object
object
Keys not in diccionario: []
Count of keys not in diccionario: 0


## 3.2 Listado beneficiarios2023

Esta sección se encarga de completar el listado original de Beneficiarios Autorizados con los nombre corregido de INEGI usando el diccionario.

In [59]:
# Crear una variable KEY en listado de productores y el diccionario para hacer el join

# Clean listado beneficiarios
listado_beneficiarios['ESTADO_Clean'] = listado_beneficiarios['ENTIDAD'].apply(clean_text)
listado_beneficiarios['MUNICIPIO_Clean'] = listado_beneficiarios['MUNICIPIO'].apply(clean_text)

# Create KEY in listado beneficiarios
listado_beneficiarios['Estado-mun-KEY'] = listado_beneficiarios['ESTADO_Clean'].astype(str) + '-' + listado_beneficiarios[
    'MUNICIPIO_Clean'].astype(str)

In [60]:
nan_rows = listado_beneficiarios.isna()

In [61]:
nan_rows.sum()

BENEFICIARIO       0
ZONA               0
ENTIDAD            0
MUNICIPIO          0
LOCALIDAD          0
ESTRATIFICACIÓN    0
PRODUCTO           0
FECHA              0
MONTO FEDERAL      0
CICLO AGRÍCOLA     0
ESTADO_Clean       0
MUNICIPIO_Clean    0
Estado-mun-KEY     0
dtype: int64

In [62]:
print(listado_beneficiarios.shape)
listado_beneficiarios.columns


(1869033, 13)


Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PRODUCTO', 'FECHA', 'MONTO FEDERAL',
       'CICLO AGRÍCOLA', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY'],
      dtype='object')

In [63]:
# Lectura del diccionario Verificado Simple
#diccionario_verificado = pd.read_csv('../../data/Diccionario_benef_manual_verificado.csv')
diccionario_verificado_e3 = pd.read_csv('../../data/DiccionarioE3_unique_mun.csv')

In [64]:
descriptive_stats = diccionario_verificado_e3.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

                    count unique                                 top freq  \
ENTIDAD              1033     10                              OAXACA  558   
MUNICIPIO            1033   1025                     EMILIANO ZAPATA    3   
ESTADO_c_benef       1033     10                              oaxaca  558   
MUNICIPIO_c_benef    1033   1025                     emiliano zapata    3   
KEY_benef            1033   1033              oaxaca-san blas atempa    1   
KEY_inegi            1033   1031         oaxaca-juchitan de zaragoza    2   
best_match           1033   1029  chiapas-san cristobal de las casas    2   
match_score        1033.0    NaN                                 NaN  NaN   
CVE_ENT            1033.0    NaN                                 NaN  NaN   
Entidad_inegi        1033      9                              Oaxaca  557   
CVE_MUN            1033.0    NaN                                 NaN  NaN   
Municipio_inegi      1033   1022                     Emiliano Zapata    3   

In [65]:
diccionario_verificado_e3.columns

Index(['ENTIDAD', 'MUNICIPIO', 'ESTADO_c_benef', 'MUNICIPIO_c_benef',
       'KEY_benef', 'KEY_inegi', 'best_match', 'match_score', 'CVE_ENT',
       'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi', 'CVE_LOC',
       'Localidad_inegi', 'Entidad_c_inegi', 'Municipio_c_inegi',
       'Unnamed: 16', 'CORRECIÓN'],
      dtype='object')

El diccionatio contiene KEY_benef únicos (Freq = 1), mientras que KEY_benef_Verificado puede repetirse.

### MERGE

Armamos por partes el dataset definitivo:
1. Juntamos Listado_beneficiarios con el diccionario simple. (listado_beneficiario_parte_I)
2. Al df anterior juntamos las claver provenientes del catálogo de Inegi. (listado_beneficiario_parte_II)

In [66]:
# Hacer el join de la Parte I
listado_beneficiarios_parte_I = pd.merge(listado_beneficiarios, diccionario_verificado_e3, left_on="Estado-mun-KEY",
                                        right_on="KEY_benef", how='left', suffixes=('_benef', '_inegi'))

In [67]:
listado_beneficiarios_parte_I.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD_benef', 'MUNICIPIO_benef', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PRODUCTO', 'FECHA', 'MONTO FEDERAL',
       'CICLO AGRÍCOLA', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY',
       'ENTIDAD_inegi', 'MUNICIPIO_inegi', 'ESTADO_c_benef',
       'MUNICIPIO_c_benef', 'KEY_benef', 'KEY_inegi', 'best_match',
       'match_score', 'CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi',
       'CVE_LOC', 'Localidad_inegi', 'Entidad_c_inegi', 'Municipio_c_inegi',
       'Unnamed: 16', 'CORRECIÓN'],
      dtype='object')

In [68]:
listado_beneficiarios_parte_I_nan_counts = listado_beneficiarios_parte_I.isna().sum()
print(listado_beneficiarios_parte_I_nan_counts)

BENEFICIARIO               0
ZONA                       0
ENTIDAD_benef              0
MUNICIPIO_benef            0
LOCALIDAD                  0
ESTRATIFICACIÓN            0
PRODUCTO                   0
FECHA                      0
MONTO FEDERAL              0
CICLO AGRÍCOLA             0
ESTADO_Clean               0
MUNICIPIO_Clean            0
Estado-mun-KEY             0
ENTIDAD_inegi              0
MUNICIPIO_inegi            0
ESTADO_c_benef             0
MUNICIPIO_c_benef          0
KEY_benef                  0
KEY_inegi                  0
best_match                 0
match_score                0
CVE_ENT                    0
Entidad_inegi              0
CVE_MUN                    0
Municipio_inegi            0
CVE_LOC                    0
Localidad_inegi            0
Entidad_c_inegi            0
Municipio_c_inegi          0
Unnamed: 16          1869033
CORRECIÓN            1868062
dtype: int64


In [69]:
# Valores únicos y la cantidad de cada columna
# Obtener estadísticas descriptivas para todas las variables

descriptive_stats = listado_beneficiarios_parte_I.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

                       count  unique                           top     freq  \
BENEFICIARIO         1869033  817427             PEREZ PEREZ JUAN       111   
ZONA                 1869033       8                   CENTRO PAÍS  1424972   
ENTIDAD_benef        1869033      10                      GUERRERO  1279305   
MUNICIPIO_benef      1869033    1025            CHILAPA DE ÁLVAREZ    60148   
LOCALIDAD            1869033    9979                     TECOANAPA    10820   
ESTRATIFICACIÓN      1869033       7                          Alto  1253286   
PRODUCTO             1869033       7  Cultivo de maíz grano blanco  1132720   
FECHA                1869033     815                    15/07/2019    17568   
MONTO FEDERAL      1869033.0     NaN                           NaN      NaN   
CICLO AGRÍCOLA       1869033       4                        PV2022   826382   
ESTADO_Clean         1869033      10                      guerrero  1279305   
MUNICIPIO_Clean      1869033    1025            chil

In [70]:
# Seleccionamos las columnas que nos interesan
listado_beneficiarios_parte_I = listado_beneficiarios_parte_I[
    ['ZONA', 'CVE_ENT', 'ENTIDAD_inegi', 'CVE_MUN', 'MUNICIPIO_inegi', 'BENEFICIARIO', 'PRODUCTO', 'FECHA',
     'MONTO FEDERAL', 'CICLO AGRÍCOLA']]

In [71]:
# Revisamos el dataset
print(listado_beneficiarios_parte_I.shape)
print(listado_beneficiarios_parte_I.columns)
print(listado_beneficiarios_parte_I.head())

(1869033, 10)
Index(['ZONA', 'CVE_ENT', 'ENTIDAD_inegi', 'CVE_MUN', 'MUNICIPIO_inegi',
       'BENEFICIARIO', 'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA'],
      dtype='object')
          ZONA  CVE_ENT ENTIDAD_inegi  CVE_MUN    MUNICIPIO_inegi  \
0  CENTRO PAÍS       12      GUERRERO       21  COYUCA DE BENÍTEZ   
1  CENTRO PAÍS       12      GUERRERO       21  COYUCA DE BENÍTEZ   
2  CENTRO PAÍS       12      GUERRERO       21  COYUCA DE BENÍTEZ   
3  CENTRO PAÍS       12      GUERRERO       21  COYUCA DE BENÍTEZ   
4  CENTRO PAÍS       12      GUERRERO       21  COYUCA DE BENÍTEZ   

              BENEFICIARIO                      PRODUCTO       FECHA  \
0  ABAC NERI JESUS CARLOS   Cultivo de maíz grano blanco  2022-06-26   
1    ABAD ALARCON ELIAZAR   Cultivo de maíz grano blanco  2022-06-26   
2   ABAD ALARCON EZEQUIEL   Cultivo de maíz grano blanco  2022-07-12   
3  ABAD ALARCON FULGENCIO   Cultivo de maíz grano blanco  2022-07-12   
4    ABAD ALARCON ROMALDA   Cultivo 

In [72]:
nan_counts = listado_beneficiarios_parte_I.isna().sum()
print(nan_counts)

ZONA               0
CVE_ENT            0
ENTIDAD_inegi      0
CVE_MUN            0
MUNICIPIO_inegi    0
BENEFICIARIO       0
PRODUCTO           0
FECHA              0
MONTO FEDERAL      0
CICLO AGRÍCOLA     0
dtype: int64
